In [ ]:
import torch.utils.tensorboard as tensorboard
run_dir = ''
stats_tfevents = tensorboard.SummaryWriter(run_dir)

### Test Dataset (adding det)

In [ ]:
from datasets import dataset_512
import importlib
importlib.reload(dataset_512)

res = 256
# dpath = '/media/nnthao/MAT/Data/CelebA-HQ/CelebA-HQ-img/'
dpath = '/home/nnthao/lntuong/FDA/test_sets/CelebA-HQ/images3'
D = dataset_512.ImageFolderMaskDataset(path=dpath, det_path='/media/nnthao/MAT/label_yolo/CelebA-HQ/CelebA-HQ-img/', xflip=True)
print(D.__len__())
for i in range(D.__len__()):
    print(i)
    a, b, c, d = D.__getitem__(i)
    if a.shape != (3, 512, 512):
        print(i, a.shape)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# a = D._load_raw_det(15)
a = D.__getitem__(16)[3].transpose(1, 2, 0)
plt.imsave('det.png', np.concatenate((a, a, a), axis=2))

### Test-case MAT+yolo

In [ ]:
import matplotlib.image as mpimg
from _my_utils import *

In [ ]:
dpath = 'test_sets/CelebA-HQ/images3/'
mpath = 'test_sets/CelebA-HQ/masks3/'
outdir = 'test_sets/CelebA-HQ/samples3/'

full_imgs = []

for i in range(8):
    full_imgs.append(mpimg.imread(dpath + str(i) + '.png')[..., :-1])
for i in range(8):
    img = mpimg.imread(dpath + str(i) + '.png')[..., :-1]
    mask = mpimg.imread(mpath + str(i) + '.png')[..., :-1]
    full_imgs.append(img * mask)
for i in range(8):
    full_imgs.append(mpimg.imread(outdir + str(i) + '.png'))

plot_images(np.array(full_imgs), 'rectangle', (3, 8))